In [10]:
import torch
from tqdm import tqdm

from gfos.model.gnn import LayoutModel
from gfos.data.dataset import LayoutDataset, Normalizer
from gfos.data.utils import load_layout

In [11]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"
NORMALIZER_PATH = "../../data/normalizer.json"

SOURCE = "nlp"
SEARCH = "default"

In [12]:
normalizer = Normalizer.from_json(NORMALIZER_PATH, SOURCE, SEARCH)

layout_data = load_layout(
    LAYOUT_DIR,
    model_type=SOURCE,
    compile_type=SEARCH,
)

test_dataset = LayoutDataset(
    layout_data["test"],
    normalizer=normalizer,
)

Loading data:   0%|          | 0/17 [00:00<?, ?it/s, 016ac66a44a906a695afd2228509046a]g:\projects\gfos\src\gfos\data\dataset.py:144: RuntimeWarning: invalid value encountered in divide
  runtime = (runtime - runtime.mean()) / runtime.std()
Loading data: 100%|██████████| 17/17 [00:01<00:00,  8.67it/s, f6c146fc5cf10be4f3accbaca9897311]


In [13]:
model1 = LayoutModel(node_feat_dim=112, node_config_dim=14, config_neighbor_layer="GATConv", dropout=0.2)
state_dict = torch.load(r"G:\projects\gfos\logs\pipeline\layout_nlp_default_20231024_020455\runs\2023-10-24_02-04-55\wandb\run-20231024_020902-b7lcp2vw\files\1119_0.4991.pth")
model1.load_state_dict(state_dict["state_dict"])

model2 = LayoutModel(node_feat_dim=112, node_config_dim=14, config_neighbor_layer="GATConv", dropout=0.2)
state_dict = torch.load(r"G:\projects\gfos\logs\pipeline\layout_nlp_default_20231017_114957\runs\2023-10-17_11-49-57\wandb\run-20231017_115319-2el0weqq\files\1239_0.5009.pth")
model2.load_state_dict(state_dict["state_dict"])


Disable config edge weight
Disable config edge weight


<All keys matched successfully>

In [14]:
model1 = model1.eval().cuda()
model2 = model2.eval().cuda()
models = [model1, model2]

In [15]:
from gfos.metrics import LayoutMetrics

In [19]:
device = "cuda"
batch_size = 300

results = {}
metric = LayoutMetrics()

with torch.no_grad():
    for record in tqdm(test_dataset):
        model_id = record["model_id"]
        node_feat = record["node_feat"]
        node_opcode = record["node_opcode"]
        edge_index = record["edge_index"]
        node_config_feat = record["node_config_feat"]
        node_config_ids = record["node_config_ids"]
        config_runtime = record["config_runtime"]
        config_edge_index = record["config_edge_index"]
        config_edge_weight = record["config_edge_weight"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            config_edge_index,
            config_edge_weight,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
            config_edge_index.to(device),
            config_edge_weight.to(device),
        )

        c = len(config_runtime)

        blend_outs = []
        for model in models:
            outs = []
            
            for i in range(0, c, batch_size):
                end_i = min(i + batch_size, c)
                out: torch.Tensor = model(
                    node_feat,
                    node_opcode,
                    edge_index,
                    node_config_feat[i:end_i],
                    node_config_ids,
                    config_edge_index,
                    config_edge_weight,
                )
                outs.append(out.detach().cpu())
            outs = torch.cat(outs)
            blend_outs.append(outs)
        outs = torch.stack(blend_outs).mean(dim=0)
        metric.add(model_id, outs.numpy(), config_runtime.numpy())
        results[model_id] = torch.argsort(outs).cpu().numpy()

100%|██████████| 17/17 [00:04<00:00,  3.58it/s]


In [ ]:
metric.compute_scores()

In [20]:
with open(f"../../output/{SOURCE}_{SEARCH}.csv", "w") as f:
    f.write("ID,TopConfigs\n")
    for k, v in results.items():
        model_id = f"layout:{SOURCE}:{SEARCH}:"+k
        values = ";".join([str(i) for i in v])
        f.write(f"{model_id},{values}\n")
